In [1]:
import nltk
import import_ipynb
from matscholar import MatScholarProcess

def sen_seg(data):    
    to_replace = ['et al. ', 'Fig. ', 'e.g. ', 'i.e. ', 'Ref. ', 'Figs. ', ' ca. ', 'approx. ', '(ca. ', 'etc.) ']
    for tr in to_replace:
        data = data.replace(tr, tr[:-2]+'####@')
    tmp = nltk.sent_tokenize(data)
    for i, t in enumerate(tmp):
        for tr in to_replace:
            t = t.replace(tr[:-2]+'####@', tr)
        tmp[i] = t
    return tmp

def process_file(text):
    zt_answer = {}
    for i, z in enumerate(text):
        z_split = z.split(', ')
        if z_split[1] != 'NaN':
            mat_name = z_split[0][1:-1]
            mat_rec = float(z_split[1])
            if mat_name in zt_answer.keys():
                if mat_rec > zt_answer[mat_name]:
                    zt_answer[mat_name] = mat_rec
            else:
                zt_answer[mat_name] = mat_rec

    sorted_zt = sorted(zt_answer.items(), key=lambda item: item[1], reverse=True)
    zt_rank = {}
    rank = 1
    processor = MatScholarProcess()
    for s in sorted_zt:
        processed, _ = processor.process(s[0])
        if processed[0] not in zt_rank.keys():
            zt_rank[processed[0]] = {}
            zt_rank[processed[0]]['rank'] = rank
            zt_rank[processed[0]]['value'] = s[1]
            rank += 1
    return zt_rank

In [2]:
import json

with open('thermo_meta.json', 'r', encoding='utf-8') as f:
    doi_meta = json.load(f)
with open('thermo.json', 'r', encoding='utf-8') as f:
    ab_dict = json.load(f)
    
for d in list(doi_meta.keys())[:1]:
    print(d, doi_meta[d])
for d in list(ab_dict.keys())[:1]:
    print(d, ab_dict[d])

10.1016/j.ijhydene.2011.09.125 {'publisher': 'PERGAMON-ELSEVIER SCIENCE LTD', 'abstract': 'Thermo-Electric (TE) power generation is an attractive method for the direct conversion of thermal energy into electrical one. Both the TE material properties and the power generator architecture play a fundamental role in the achievement of high energy conversion efficiencies. The paper focuses on the TE module architecture design, as well as on the sub-components manufacturing and assembly.', 'journal': 'Int. J. Hydrog. Energy', 'year': '2012', 'type': 'Article', 'category': 'Chemistry; Electrochemistry; Energy & Fuels', 'citation': '25', 'wc': 'Chemistry, Physical; Electrochemistry; Energy & Fuels', 'i_d': 'CATALYTIC COMBUSTION; HYDROGEN-AIR; SYSTEM; MONOLITHS; MIXTURES;', 'title': 'High efficiency Thermo-Electric power generator', 'keywords': 'Thermo-Electric materials; Power generation; Seebeck effect;'}
10.1016/j.ijhydene.2011.09.125 Thermo-Electric (TE) power generation is an attractive me

In [3]:
print(len(doi_meta))
print(len(ab_dict))

57158
52981


In [3]:
with open('PF.txt', 'r', encoding='utf-8') as f:
    pf = str(f.readlines())
pf_split = pf[4:-4].split('], [')
pf_rank = process_file(pf_split)

In [6]:
from chemdataextractor import Document
from tqdm import tqdm


mat_sen = {}
sen_meta = {}
processor = MatScholarProcess()
for d in tqdm(list(ab_dict.keys())):
    ab_sens = sen_seg(ab_dict[d])
    for s in ab_sens:
        s_cems = Document(s).cems
        for sc in s_cems:
            processed, _ = processor.process(sc.text)
            if len(processed)==1:
                if len(_) == 1:
                    nom = _[0][1]
                else:
                    nom = processed[0]
            else:
                if len(_)!=0:
                    nom = _[-1][1]
                else:
                    nom = ' '.join(processed)
            if sc.text == 'bismuth telluride':
                nom = 'Bi2Te3'
            if sc.text == 'perovskites':
                nom = 'perovskite'
            if nom in pf_rank.keys():
                if s not in sen_meta.keys():
                    sen_meta[s] = {}
                    sen_meta[s]['doi'] = d
                    if 'year' in doi_meta[d].keys():
                        sen_meta[s]['year'] = doi_meta[d]['year']
                    else:
                        sen_meta[s]['year'] = '?'  
            if nom not in mat_sen.keys():
                mat_sen[nom] = [s]
            else:
                mat_sen[nom].append(s)      


100%|██████████| 52981/52981 [1:10:53<00:00, 12.46it/s]


In [8]:
import kashgari
from kashgari.tasks.classification import CNN_Attention_Model
import json
from collections import Counter

loaded_model = CNN_Attention_Model.load_model('CNN_Attention_91')
for_predict = []
mats = []
# for o in sen_meta.keys():
    # for_predict.append(o.split(' '))
for o in mat_sen.keys():
    if o in pf_rank.keys():
        mats.append(o)
        for s in mat_sen[o]:
            for_predict.append(s.split(' '))
results = loaded_model.predict(for_predict)
# for i, o in enumerate(sen_meta.keys()):
    # sen_meta[o]['attitude'] = results[i]

2022-04-05 23:34:56,423 [DEBUG] kashgari - ------------------------------------------------
2022-04-05 23:34:56,458 [DEBUG] kashgari - Loaded gensim word2vec model's vocab
2022-04-05 23:34:56,460 [DEBUG] kashgari - model        : mat_embedding3/vectors.txt
2022-04-05 23:34:56,462 [DEBUG] kashgari - word count   : 529690
2022-04-05 23:34:56,465 [DEBUG] kashgari - Top 50 words : ['the', 'of', '.', ',', 'and', '<nUm>', 'in', 'a', 'to', ')', '(', 'with', '-', 'for', 'is', 'by', 'on', 'was', 'at', 'were', 'that', '–', 'as', 'are', 'from', '/', 'an', 'temperature', 'surface', 'using', 'high', 'which', 'C', '°', 'this', '%', 'In', 'it', 'A', '=', 'structure', 'properties', ':', 'phase', 'results', 'effect', 'these', 'than', 'based', 'different']
2022-04-05 23:34:56,467 [DEBUG] kashgari - ------------------------------------------------
2022-04-05 23:35:04,108 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 304
2022-04-05 23:35:04,453 [DEBUG] kash

In [12]:
pf_mat_sen = {}
pf_sen_info = {}
for m in mats:
    pf_mat_sen[m] = list(set(mat_sen[m]))
for i, p in enumerate(for_predict):
    p_ = ' '.join(p)
    pf_sen_info[p_] = sen_meta[p_]
    pf_sen_info[p_]['attitude'] = results[i]

In [13]:
json_str = json.dumps(pf_mat_sen, indent=4)
with open('pf_mat_sen.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)
json_str = json.dumps(pf_sen_info, indent=4)
with open('pf_sen_info.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

In [4]:
import json

material_83 = ['Bi2Te3', 'Bi23Sb2', 'Ca50Mn49NbO150', 'Hf7Ni20Sn20Ti6Zr7', 'HfNi4Sn4Ti2Zr', 'Co20InSb60', 'AlCa3Sb3', 'NiSnTi', 'AlO50Zn49', 'Co20La19O60Sr', 'Bi9Sb', 'Sb2Te3', 'Ca3Co4O9', 'Co40InSb120', 'HfNi2Sn2Zr', 'Bi4CsTe6', 'O1361W500', 'Bi22Sb3', 'NiSnZr', 'CeCoFe3Sb12', 'Bi8K2Se13', 'MnSb11Yb14', 'Co2NaO4', 'Sb3Zn4', 'Cu2Se', 'Ba4Ga8Ge15', 'Ba4Ga8Sn15', 'Mo3NiS4', 'Ni50SbSn49Zr50', 'Ba4Ga8Si15', 'Al2Ca5Sb6', 'BiTe6Tl9', 'CaMnO3', 'CoFe3LaSb12', 'GeSi4', 'Ce2CoFe7Sb24', 'LaO60Sr19Ti20', 'Cu99Se50', 'Ca9Mn10O30Yb', 'Mo3Te4', 'NbO15Sr5Ti4', 'Mg2Si', 'LaO30Sr9Ti10', 'Ga8Ge15Sr4', 'Bi83Sb17', 'Ca2Co2O5', 'CuO2Rh', 'SnTe5Tl2', 'AgCrSe2', 'BiCoO4Sr', 'LaO15Sr4Ti5', 'Ca9CeMn10O30', 'O29W10', 'CoLaO3', 'GeSi9', 'CeFe4Sb12', 'NiO50Zn49', 'Cr5Se8Tl', 'Cu10La19O40Sr', 'In2O3', 'CaCuMn6O12', 'Cu2Mo3S4', 'Bi2O7Ru2', 'Cr19Cu20MgO40', 'Cu20La37O80Sr3', 'CrCuO2', 'AlO20Zn19', 'LiMn2O4', 'OZn', 'Ba39Nb200O600Sr61', 'BaO3Pb', 'LaNiO3', 'O3W', 'O7Ru2Y2', 'CaMn2O6Sm', 'Ca9LaMn10O30', 'Nd2O7Ru2', 'CuLa2O4', 'CaMn7O12', 'Co5La4O15Sr', 'CaMn2O6Pr', 'NiO', 'CuNd2O4']
with open('pf_mat_sen.json', 'r', encoding='utf-8') as f:
    mat_sen = json.load(f)
# key = mat, value = sens
with open('pf_sen_info.json', 'r', encoding='utf-8') as f:
    sen_info = json.load(f)          
# key = s, value = doi, year, attitude    

In [14]:
pf_sen_record = {}
# key = m, value = {time: {driver:[sens]; barrier:[sens]}}
for m in mat_sen.keys():
    pf_sen_record[m] = {}
    for s in mat_sen[m]:
        time = sen_info[s]['year']
        att = sen_info[s]['attitude']
        if time not in pf_sen_record[m].keys():
            pf_sen_record[m][time] = {}
            pf_sen_record[m][time][att] = [s]
        else:
            if att not in pf_sen_record[m][time].keys():
                pf_sen_record[m][time][att] = [s]
            else:
                pf_sen_record[m][time][att].append(s)
for p in pf_sen_record:
    print(p, pf_sen_record[p])

OZn {'2012': {'barrier': ['This paper reports the synthesis, crystal structure and electrical conductivity properties of vanadium (V)-doped zinc oxide (ZnO) powders (i.e. Zn1-2X V (X) O binary system, x = 0, 0.0025, 0.005, 0.0075 and in the range 0.01 a parts per thousand currency sign x a parts per thousand currency sign 0.15).', 'The thermoelectric elements consist of ZnO (n-type) and CuO (p-type) bundles of quasi-monodimensional nanowires deposited utilizing shadow masks.', 'The lowest resistivity of the ZnO:In thin films 6.1x10(-4) Omega.cm occurs at 5at.% in the target doping concentration.', '2 mol% Al-doped ZnO nanoparticles were consolidated into a ZnO nanocomposite with ZnAl2O4 nanoprecipitates by spark plasma sintering and its high-temperature charge transport and thermoelectric properties were investigated up to 1073 K. The carrier concentration in the nanocomposite was not dependent on the temperature, while the Hall mobility showed positive temperature-dependence due to gr

In [15]:
json_str = json.dumps(pf_sen_record, indent=4)
with open('pf_sen_record.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

In [15]:
material_our = {}
for m in material_83:
    if m in mat_sen.keys():
        sens = mat_sen[m]
        for s in sens:
            year = sen_info[s]['year']
            attitude = sen_info[s]['attitude']
            if m not in material_our.keys():
                material_our[m] = {}
                material_our[m][year] = {}
                material_our[m][year][attitude] = 1
            else:
                if year not in material_our[m].keys():
                    material_our[m][year] = {}
                    material_our[m][year][attitude] = 1
                else:
                    if attitude not in material_our[m][year].keys():
                        material_our[m][year][attitude] = 1
                    else:
                        material_our[m][year][attitude] += 1
print(len(material_our.keys()))
json_str = json.dumps(material_our, indent=4)
with open('material_thermo_54.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

54


In [156]:
def cal_d(p_, d_):
    if p_ != 0:
        d_ = d_/p_
    else:
        d_ = 0
    return d_
# p_10, p11_13, p14_16, p17_19, p20_21 
fre_dis = {}
for m in mat_sen.keys():
    p_10, p11_13, p14_16, p17_19, p20_21 = 0, 0, 0, 0, 0
    d_10, d11_13, d14_16, d17_19, d20_21 = 0, 0, 0, 0, 0  
    d = 0
    p_t = len(mat_sen[m])
    for s in mat_sen[m]:  
        if sen_meta[s]['attitude'] == 'driver':
            d += 1
        if sen_meta[s]['year'] != '?':
            t = sen_meta[s]['year']            
            if int(t)<2011:
                p_10 += 1
                if sen_meta[s]['attitude'] == 'driver':
                    d_10 += 1
            elif int(t)<2014:
                p11_13 += 1
                if sen_meta[s]['attitude'] == 'driver':
                    d11_13 += 1
            elif int(t)<2017:
                p14_16 += 1
                if sen_meta[s]['attitude'] == 'driver':
                    d14_16 += 1
            elif int(t)<2020:
                p17_19 += 1
                if sen_meta[s]['attitude'] == 'driver':
                    d17_19 += 1
            elif int(t)<2023:
                p20_21 += 1
                if sen_meta[s]['attitude'] == 'driver':
                    d20_21 += 1
    d_10 = cal_d(p_10, d_10)
    d11_13 = cal_d(p11_13, d11_13)
    d14_16 = cal_d(p14_16, d14_16)
    d17_19 = cal_d(p17_19, d17_19)
    d20_21 = cal_d(p20_21, d20_21)
    p_10 = cal_d(len(mat_sen[m]), p_10)
    p11_13 = cal_d(len(mat_sen[m]), p11_13)
    p14_16 = cal_d(len(mat_sen[m]), p14_16)
    p17_19 = cal_d(len(mat_sen[m]), p17_19)
    p20_21 = cal_d(len(mat_sen[m]), p20_21)
    d = cal_d(len(mat_sen[m]), d)
    fre_dis[m] = {'p_t':p_t, 'p_10': p_10, 'p11_13': p11_13, 'p14_16':p14_16, 'p17_19':p17_19, 'p20_21':p20_21,
                  'd': d, 'd_10': d_10, 'd11_13': d11_13, 'd14_16': d14_16, 'd17_19': d17_19, 'd20_21': d20_21}

json_str = json.dumps(fre_dis, indent=4)
with open('fre_dis.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)           

In [148]:
with open('embbeding_sims.json', 'r', encoding='utf-8') as f:
    sims = json.load(f)

sim_dict = {}
rank = 1
for s in sims:
    if s[0] in pf_rank.keys():
        sim_dict[s[0]] = {}
        sim_dict[s[0]]['rank'] = rank
        sim_dict[s[0]]['value'] = s[1]
        rank += 1

In [154]:
from sklearn import linear_model
from sklearn import preprocessing as spp

add = {}
for k in fre_dis['Bi2Te3'].keys():
    x_data = []
    y_train_data = []
    x_predict_data = []
    predict_sample = []
    for s in sim_dict.keys():
        if s in fre_dis.keys():
            x_data.append([sim_dict[s]['rank'], sim_dict[s]['value']])
            y_train_data.append(fre_dis[s][k])
        else:
            x_predict_data.append([sim_dict[s]['rank'], sim_dict[s]['value']])
            predict_sample.append(s)
    scaler = spp.MinMaxScaler()
    x_train_data = scaler.fit_transform(x_data)
    reg = linear_model.LinearRegression()
    reg.fit(x_train_data, y_train_data)
    predict_result = reg.predict(x_train_data)
    for i, m in enumerate(predict_sample):
        if m not in add.keys():
            add[m] = {}
            add[m][k] = predict_result[i]
        else:
            add[m][k] = predict_result[i]

In [155]:
for a in add.keys():
    if a not in fre_dis.keys():
        # add[a]['p_t'] = add[a]['p_t']/100
        p_list = [add[a]['p_10'], add[a]['p11_13'], add[a]['p14_16'], add[a]['p17_19'], add[a]['p20_21']]
        d_list = [add[a]['d_10'], add[a]['d11_13'], add[a]['d14_16'], add[a]['d17_19'], add[a]['d20_21']]
        for k in ['p_10', 'p11_13', 'p14_16', 'p17_19', 'p20_21']:
            add[a][k] = (add[a][k] - min(p_list)) / (max(p_list)-min(p_list))
        for k in ['d_10', 'd11_13', 'd14_16', 'd17_19', 'd20_21']:
            add[a][k] = (add[a][k] - min(d_list)) / (max(d_list)-min(d_list))
        fre_dis[a] = add[a]
json_str = json.dumps(fre_dis, indent=4)
with open('fre_dis.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str) 